#**Amir Golan (204327050), Nadav Loebl (203766985)**

Cloning the data

In [2]:
!git clone https://github.com/hexiangnan/neural_collaborative_filtering.git

Cloning into 'neural_collaborative_filtering'...
remote: Enumerating objects: 66, done.
remote: Total 66 (delta 0), reused 0 (delta 0), pack-reused 66
Unpacking objects: 100% (66/66), done.


In [1]:
!ls neural_collaborative_filtering/Data

'ls' is not recognized as an internal or external command,
operable program or batch file.


Reading the csv's

In [4]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

names = ['user_id', 'item_id', 'rating', 'timestamp']
training = pd.read_csv('./neural_collaborative_filtering/Data/ml-1m.train.rating', '\t', names=names)
test_rating = pd.read_csv('./neural_collaborative_filtering/Data/ml-1m.test.rating', '\t', names=names)
negative_ids = []
negative_ids.append(f'(user_id, item_id)')
for i in range(1,100):
  negative_ids.append(f'id-{i}')
test_negative = pd.read_csv('./neural_collaborative_filtering/Data/ml-1m.test.negative', '\t', names=negative_ids)

#1.a

In [5]:
training.head()

,user_id,item_id,rating,timestamp
0,0,32,4,978824330
1,0,34,4,978824330
2,0,4,5,978824291
3,0,35,4,978824291
4,0,30,4,978824291


In [6]:
training.describe()

,user_id,item_id,rating,timestamp
count,994169.000000,994169.000000,994169.000000,9.941690e+05
mean,3023.536725,872.860828,3.581378,9.722116e+08
std,1728.320003,738.213243,1.116791,1.208143e+07
min,0.000000,0.000000,1.000000,9.567039e+08
25%,1505.000000,258.000000,3.000000,9.653021e+08
50%,3071.000000,690.000000,4.000000,9.729723e+08
75%,4475.000000,1283.000000,4.000000,9.752182e+08
max,6039.000000,3705.000000,5.000000,1.046455e+09


So, ml-1m.train.rating is the training file. Each line is a training instance tab (\t) sepeated of: userID, itemID, rating, timestamp (if have). This file is similar to the original ml-1m training files, the difference is that it contains only users with at least 20 ratings (to avoid situation where a user has only one rating). 

In [7]:
test_rating.head()

,user_id,item_id,rating,timestamp
0,0,25,5,978824351
1,1,133,3,978300174
2,2,207,4,978298504
3,3,208,4,978294282
4,4,222,2,978246585


In [8]:
test_rating.describe()

,user_id,item_id,rating,timestamp
count,6040.000000,6040.000000,6040.000000,6.040000e+03
mean,3019.500000,939.856623,3.612252,9.775217e+08
std,1743.742145,789.260796,1.166819,2.008596e+07
min,0.000000,0.000000,1.000000,9.567124e+08
25%,1509.750000,277.000000,3.000000,9.654912e+08
50%,3019.500000,741.000000,4.000000,9.745024e+08
75%,4529.250000,1400.000000,5.000000,9.764266e+08
max,6039.000000,3573.000000,5.000000,1.046455e+09


Now,  ml-1m.test.rating is the (rating) test file for positives instances. Each line is a testing instance tab (\t) sepeated of: userID, itemID, rating, timestamp (if have). The difference is that here we have only one rating for each user. 

In [9]:
test_negative.head()

,"(user_id, item_id)",id-1,id-2,id-3,id-4,id-5,id-6,id-7,id-8,id-9,id-10,id-11,id-12,id-13,id-14,id-15,id-16,id-17,id-18,id-19,id-20,id-21,id-22,id-23,id-24,id-25,id-26,id-27,id-28,id-29,id-30,id-31,id-32,id-33,id-34,id-35,id-36,id-37,id-38,id-39,...,id-60,id-61,id-62,id-63,id-64,id-65,id-66,id-67,id-68,id-69,id-70,id-71,id-72,id-73,id-74,id-75,id-76,id-77,id-78,id-79,id-80,id-81,id-82,id-83,id-84,id-85,id-86,id-87,id-88,id-89,id-90,id-91,id-92,id-93,id-94,id-95,id-96,id-97,id-98,id-99
0,"(0,25)",1064,174,2791,3373,269,2678,1902,3641,1216,915,3672,2803,2344,986,3217,2824,2598,464,2340,1952,1855,1353,1547,3487,3293,1541,2414,2728,340,1421,1963,2545,972,487,3463,2727,1135,3135,128,...,1010,1423,2840,1770,881,1913,1803,1734,3326,1617,224,3352,1869,1182,1331,336,2517,1721,3512,3656,273,1026,1991,2190,998,3386,3369,185,2822,864,2854,3067,58,2551,2333,2688,3703,1300,1924,3118
1,"(1,133)",1072,3154,3368,3644,549,1810,937,1514,1713,2186,660,2303,2416,670,1176,788,889,3120,2344,2525,3301,2055,1436,2630,11,2773,2176,1847,740,2332,3561,263,3658,3282,1980,2093,3287,3190,3475,...,2976,918,753,3540,3341,2973,1580,2118,3511,526,1719,525,1520,486,557,1353,500,2902,1687,1295,2997,2415,797,2518,926,3537,1746,1676,1875,3029,1535,341,3525,1429,2225,1628,2061,469,3056,2553
2,"(2,207)",2216,209,2347,3,1652,3397,383,2905,2284,2866,584,783,3208,1534,2529,1907,1170,3037,2015,1045,3099,3298,2522,739,2652,3702,792,2527,1945,2333,1668,3511,70,1991,3071,2474,1629,3221,505,...,1906,1812,1182,2831,1548,1630,2227,2352,760,350,302,791,300,3528,1444,2,798,997,376,2565,1565,718,710,2695,904,3643,655,3666,3069,3661,953,865,813,1353,2945,2580,2989,2790,2879,2481
3,"(3,208)",3023,1489,1916,1706,1221,1191,2671,81,2483,941,841,1617,1437,2700,1904,1763,1181,599,2442,1656,1370,1171,1372,1444,1596,2023,1456,2179,530,1949,2628,290,1996,2593,2489,3484,1035,2826,1274,...,3655,3073,557,2007,1937,2244,55,1016,2382,2506,3501,914,3127,23,3187,799,2572,1038,3028,2619,1429,2623,2158,2785,3674,2578,1837,1689,296,959,3347,1707,2901,2767,2167,1921,247,1618,2016,2323
4,"(4,222)",1794,3535,108,593,466,2048,854,1378,1301,697,1376,22,2135,3657,3173,1322,976,6,1399,817,2757,2010,652,458,1227,204,592,2875,1930,2251,1654,2542,1149,3089,454,2466,3461,1770,3106,...,3610,959,1927,2524,932,3327,187,2575,1674,557,2547,1572,776,1600,2682,2085,1987,1390,614,3098,1831,927,2285,1059,2850,3517,134,1852,2776,1694,2490,1332,2526,2804,2027,833,176,463,2851,2453


The ml-1m.test.negative file is the test file for negative instances. Each line corresponds to the line of test.rating, containing 99 negative samples.
Each line is in the format: (userID,itemID)\t negativeItemID1\t negativeItemID2 ...negativeItemID99. <br>
For example, as we can see the first instance of test.rating is of user_id=0, item_id=25. For this pair we keep 99 negative items id's. Also here we will get that each user will only have one feedback record (followed by 99 negatives items ids).

# 1.b

The training dataset is created as following: <br> First we set num_negatives parameter. This parameter will define how many negative examples we add to our training set for each existing rating. <br> We initialize some empty matrix A. Then, we are iterating throguh our ratings matrix and for each user u that rated an item i: <br>
* We add the corresponding positive instance for A, that is, we add (u, i, 1) (rating ~ positive interaction).
* We are randomly selecting num_negatives items that the user u has *not* rated. For each such item j, we add the instance (u, j, 0) to A (movie that was not rated by a user ~ negative interaction).

#1.c
The testing data set is created in the same manner, only for the testing data frame.

#2.a

Creating the implicit feedback training instances from the training set

In [10]:
num_items = len(training.item_id.unique())

In [11]:
def get_train_instances(train, num_negatives):
    user_input, item_input, labels = [],[],[]
    num_users = train.shape[0]
    for i in range(len(train)):
        if i % 50000 == 0:
          print(i)
        user = train.iloc[i].user_id
        item = train.iloc[i].item_id
        # positive instance
        user_input.append(user)
        item_input.append(item)
        labels.append(1)
        # negative instances
        for t in range(num_negatives):
            j = np.random.randint(num_items)
            while not training[(training['user_id'] == user) & (training['item_id'] == j)].empty:
                j = np.random.randint(num_items)
            user_input.append(user)
            item_input.append(j)
            labels.append(0)
    return user_input, item_input, labels

In [12]:
train_instances = get_train_instances(training, 4)

0
50000
100000
150000
200000
250000
300000
350000
400000
450000
500000
550000
600000
650000
700000
750000
800000
850000
900000
950000


In [13]:
users, items, labels = train_instances

Writing the implicit feedback training data to a csv

In [14]:
with open(f'training_data.csv', 'w') as f:
    for i in range(len(users)):
        if i % 200000 == 0:
          print(i)
        f.write(f'{users[i]}\t{items[i]}\t{labels[i]}\n')

0
200000
400000
600000
800000
1000000
1200000
1400000
1600000
1800000
2000000
2200000
2400000
2600000
2800000
3000000
3200000
3400000
3600000
3800000
4000000
4200000
4400000
4600000
4800000


In [15]:
implicit_training_data = pd.read_csv('./training_data.csv', '\t', names=['user_id', 'item_id', 'label'])

In [16]:
implicit_training_data.shape

(4970845, 3)

In [17]:
implicit_training_data.head()

,user_id,item_id,label
0,0,32,1
1,0,2188,0
2,0,2026,0
3,0,1859,0
4,0,3452,0


In [18]:
users, items, labels = list(implicit_training_data['user_id'].to_numpy()), list(implicit_training_data['item_id'].to_numpy()), list(implicit_training_data['label'].to_numpy())

Defining the four models: MF, GMF, MLP and NMF

In [19]:
# Learning Functions

def gd(Y, U, V, reg, k, lr):
    m = data['user_id']
    n = data['item_id']
    per = np.random.permutation(range(m.shape[0]))

    for index in per:
        i = m[index] - 1
        j = n[index] - 1
        grad_U = (reg * U[:,i]) - (V[:,j] * np.transpose(Y[i,j] - np.dot(np.transpose(U[:,i]), V[:,j])))
        grad_V = (reg * V[:,j]) - (U[:,i] * np.transpose(Y[i,j] - np.dot(np.transpose(U[:,i]), V[:,j])))
        U[:,i] -= lr * grad_U
        V[:,j] -= lr * grad_V
        
    return U, V

def als(Y, fixed, k, reg):
  A = fixed.dot(fixed.T) + np.eye(k) * reg
  b = fixed.dot(Y.T)
  A_inv = np.linalg.inv(A)
  solved = b.T.dot(A_inv)
  return solved.T


# Loss Functions

def rmse_loss(Y, U, V):    
    m = Y.shape[0]
    n = Y.shape[1]
    index = (Y != 0)
    Y_hat = np.dot(np.transpose(U), V)
    return (1 / (m * n)) * np.sum(np.square(Y[index] - Y_hat[index]))

def mrr_score(Y, U, V, threshold=3, cutoff=5):
  index = (Y != 0)
  Y_hat= np.dot(np.transpose(U), V)
  _mrr = []
  for user in range(len(Y_hat)):
    _top_ratings_indexes = [i[0] for i in sorted(enumerate(Y_hat[user]), key=lambda x:x[1], reverse=True)]
    idx = 1
    for item in _top_ratings_indexes[:cutoff]:
      if Y[user, data['item_id'][item]-1] >= threshold:
        _mrr.append(1/idx)
      idx += 1
  return np.mean(_mrr)

def dcg(ratings):
    dcg = []
    for idx, val in enumerate(ratings): 
        numerator = 2**val - 1
        denominator =  np.log2(idx + 2) 
        score = numerator / denominator
        dcg.append(score)
    return sum(dcg)

def ndcg(Y, U, V, cutoff=10):
    Y_hat= np.dot(np.transpose(U), V)
    ndcgs = []
    for user in range(len(Y_hat)):
      _items = []
      _top_ratings_indexes = [i[0] for i in sorted(enumerate(Y_hat[user]), key=lambda x:x[1], reverse=True)]
      for item in _top_ratings_indexes[:cutoff]:
        _items.append(Y[user, data['item_id'][item]-1])
      _dcg = dcg(_items)
      sorted_items = sorted(_items)
      idcg = dcg(sorted_items)
      if idcg > 0:
        ndcg = _dcg / idcg
        ndcgs.append(ndcg)
    return np.mean(ndcgs) # average score

In [20]:
def MF(learning_func, score_loss_func, k, reg, lr, num_epochs, test):
    U = np.random.rand(k, num_users)
    V = np.random.rand(k, num_items)

    # missing samples is zero
    Y = np.zeros([num_users, num_items], dtype = 'float64')
    Y_test = np.zeros([num_users, num_items], dtype = 'float64')

    for i in range(0, len(data)):
        Y[data['user_id'][i]-1, data['item_id'][i]-1] = data['rating'][i]

    for i in range(0, len(test)):
      Y_test[test['user_id'][i]-1, test['item_id'][i]-1] = test['rating'][i]
      Y[test['user_id'][i]-1, test['item_id'][i]-1] = 0.0 # delete the test from train data

    scores_losses = []

    for e in range(1, num_epochs + 1):
        score_loss = 0
        if learning_func == 'gd':
          U, V = gd(Y, U, V, reg, k, lr)
        if learning_func == 'als':
          U = als(Y, V, k, reg)
          V = als(Y.T, U, k, reg)
        score_loss = score_loss_func(Y, U, V)
        scores_losses.append(score_loss)

    test_score_loss = score_loss_func(Y_test, U, V)
    return test_score_loss, scores_losses

In [21]:
from keras.layers import Embedding, Input, Dense, Reshape,  Flatten, Dropout
from keras.regularizers import l2
from keras import backend as K
from keras import initializers
from keras.initializers import RandomNormal
from keras.models import Sequential, Model, load_model, save_model
from keras.layers.core import Dense, Lambda, Activation
from keras.optimizers import Adagrad, Adam, SGD, RMSprop
from keras.layers import Multiply, Concatenate

def get_GMF_model(num_users, num_items, latent_dim, regs=[[0,0]]):
    #Generalized Matrix Factorization
    
    # Input variables
    user_input = Input(shape=(1,), dtype='int32', name = 'user_input')
    item_input = Input(shape=(1,), dtype='int32', name = 'item_input')

    MF_Embedding_User = Embedding(input_dim = num_users, output_dim = latent_dim, name = 'user_embedding',
                                   embeddings_regularizer = l2(regs[0][0]), input_length=1,embeddings_initializer=RandomNormal(mean=0.0, stddev=0.01)) #init = init_normal,
    MF_Embedding_Item = Embedding(input_dim = num_items, output_dim = latent_dim, name = 'item_embedding',
                                   embeddings_regularizer = l2(regs[0][1]), input_length=1,embeddings_initializer=RandomNormal(mean=0.0, stddev=0.01))  #init = init_normal, 
    
    # Crucial to flatten an embedding vector!
    user_latent = Flatten()(MF_Embedding_User(user_input))
    item_latent = Flatten()(MF_Embedding_Item(item_input))
    
    # Element-wise product of user and item embeddings 
    predict_vector = Multiply()([user_latent, item_latent]) #merge([user_latent, item_latent], mode = 'mul')
    
    # Final prediction layer
    prediction = Dense(1, activation='sigmoid', kernel_initializer='lecun_uniform', name = 'prediction')(predict_vector)
    
    model = Model(inputs=[user_input, item_input], outputs=prediction)
    return model


def get_MLP_model(num_users, num_items, latent_dim, regs=[[0,0],0,0], layers = [20,10]):
    #Multi-Layer Perceptron
    
    assert len(layers) + 1 == len(regs), 'the number of regs is equal to number of layers + the embedding layer'
    num_layer = len(layers) #Number of layers in the MLP
    # Input variables
    user_input = Input(shape=(1,), dtype='int32', name = 'user_input')
    item_input = Input(shape=(1,), dtype='int32', name = 'item_input')

    MLP_Embedding_User = Embedding(input_dim = num_users, output_dim = latent_dim, name = 'user_embedding',
                                   embeddings_regularizer = l2(regs[0][0]), input_length=1,embeddings_initializer=RandomNormal(mean=0.0, stddev=0.01)) #init = init_normal,
    MLP_Embedding_Item = Embedding(input_dim = num_items, output_dim = latent_dim, name = 'item_embedding',
                                   embeddings_regularizer = l2(regs[0][1]), input_length=1,embeddings_initializer=RandomNormal(mean=0.0, stddev=0.01)) #init = init_normal,
    
    # Crucial to flatten an embedding vector!
    user_latent = Flatten()(MLP_Embedding_User(user_input))
    item_latent = Flatten()(MLP_Embedding_Item(item_input))
    
    # Concatenation of embedding layers
    vector = Concatenate(axis=-1)([user_latent, item_latent])#merge([user_latent, item_latent], mode = 'concat')
    
    # MLP layers
    for idx in range(num_layer):
        layer = Dense(layers[idx], kernel_regularizer = l2(regs[idx+1]), activation='relu', name = 'layer%d' %idx)
        vector = layer(vector)
        
    # Final prediction layer
    prediction = Dense(1, activation='sigmoid', kernel_initializer='lecun_uniform', name = 'prediction')(vector)
    model = Model(inputs=[user_input, item_input], outputs=prediction)
    return model

def get_NMF_model(num_users, num_items, latent_dim_GMF, latent_dim_MLP, reg_GMF=[[0,0]], regs_MLP=[[0,0],0,0], layers=[20,10]):
    #Neural matrix factorization
    assert len(layers) + 1 == len(regs_MLP), 'the number of regs is equal to number of layers + the embedding layer'
    num_layer = len(layers) #Number of layers in the MLP

    # Input variables
    user_input = Input(shape=(1,), dtype='int32', name = 'user_input')
    item_input = Input(shape=(1,), dtype='int32', name = 'item_input')
    
    # Embedding layer
    MF_Embedding_User = Embedding(input_dim = num_users, output_dim = latent_dim_GMF, name = 'MF_user_embedding',
                                   embeddings_regularizer = l2(reg_GMF[0][0]), input_length=1,embeddings_initializer=RandomNormal(mean=0.0, stddev=0.01)) #init = init_normal,
    MF_Embedding_Item = Embedding(input_dim = num_items, output_dim = latent_dim_GMF, name = 'MF_item_embedding',
                                   embeddings_regularizer = l2(reg_GMF[0][1]), input_length=1,embeddings_initializer=RandomNormal(mean=0.0, stddev=0.01))  #init = init_normal, 
    
    MLP_Embedding_User = Embedding(input_dim = num_users, output_dim = latent_dim_MLP, name = 'MLP_user_embedding',
                                   embeddings_regularizer = l2(regs_MLP[0][0]), input_length=1,embeddings_initializer=RandomNormal(mean=0.0, stddev=0.01)) #init = init_normal,
    MLP_Embedding_Item = Embedding(input_dim = num_items, output_dim = latent_dim_MLP, name = 'MLP_item_embedding',
                                   embeddings_regularizer = l2(regs_MLP[0][1]), input_length=1,embeddings_initializer=RandomNormal(mean=0.0, stddev=0.01)) #init = init_normal,
    
    # MF part
    mf_user_latent = Flatten()(MF_Embedding_User(user_input))
    mf_item_latent = Flatten()(MF_Embedding_Item(item_input))
    mf_vector = Multiply()([mf_user_latent, mf_item_latent]) #merge([mf_user_latent, mf_item_latent], mode = 'mul') # element-wise multiply

    # MLP part 
    mlp_user_latent = Flatten()(MLP_Embedding_User(user_input))
    mlp_item_latent = Flatten()(MLP_Embedding_Item(item_input))
    mlp_vector = Concatenate(axis=-1)([mlp_user_latent, mlp_item_latent])#merge([mlp_user_latent, mlp_item_latent], mode = 'concat')
    for idx in range(num_layer):
        layer =  Dense(layers[idx], kernel_regularizer = l2(regs_MLP[idx+1]), activation='tanh', name = 'layer%d' %idx)
        mlp_vector = layer(mlp_vector)

    # Concatenate MF and MLP parts
    predict_vector = Concatenate(axis=-1)([mf_vector, mlp_vector])
    
    # Final prediction layer
    prediction = Dense(1, activation='sigmoid', kernel_initializer='lecun_uniform', name = "prediction")(predict_vector)    
    model = Model(inputs=[user_input, item_input], outputs=prediction)
    
    return model

#2.b

Defining some global variables

In [22]:
num_factors = 8 #size of embedding size. Can be split to 4 different params potentially.
num_negatives = 4 #how many negative samples per positive sample?
learning_rate = 0.001
batch_size = 256
verbose = 1

topK = 10 #used to evaluate the model. Top K recommendations are used. 
num_users = len(implicit_training_data.user_id.unique())
num_items = len(implicit_training_data.item_id.unique())

Defining evaluation utilities (including MRR and NDCG metrics)

In [23]:
import math
import heapq # for retrieval topK
from time import time

def evaluate_model(model, testRatings, testNegatives, K):
    """
    Evaluate the performance (MRR, NDCG) of top-K recommendation
    Return: score of each test rating.
    """ 
    mrrs, ndcgs = [],[]
    # Single thread
    for idx in range(len(testRatings)):
        (mrr,ndcg) = eval_one_rating(model, testRatings, testNegatives, idx, K)
        mrrs.append(mrr)
        ndcgs.append(ndcg)      
    return (mrrs, ndcgs)

def eval_one_rating(model, testRatings, testNegatives, idx, K):
    rating = testRatings.iloc[idx]['rating']
    u = testRatings.iloc[idx]['user_id']
    gtItem = testRatings.iloc[idx]['item_id']
    items = testNegatives[testNegatives['(user_id, item_id)'] == f'({u},{gtItem})'].to_numpy()[0]
    items = items[1:len(items)].astype('int32')
    items = np.append(items, gtItem)
    users = np.full(len(items), u, dtype = 'int32')
    # Get prediction scores
    map_item_score = {}
    predictions = model.predict([users, items], 
                                 batch_size=100, verbose=0)
    for i in range(len(items)):
        item = items[i]
        map_item_score[item] = predictions[i]
    items = items[0:len(items)-1]
    
    # Evaluate top rank list
    ranklist = heapq.nlargest(K, map_item_score, key=map_item_score.get)
    mrr = getMRR(ranklist, gtItem)
    ndcg = getNDCG(ranklist, gtItem)
    return (mrr, ndcg)


def getMRR(ranklist, gtItem):
  for i in range(0, len(ranklist)):
    if ranklist[i] == gtItem:
      return 1/(i+1)

  return 0

def getNDCG(ranklist, gtItem):
    for i in range(len(ranklist)):
        item = ranklist[i]
        if item == gtItem:
            return math.log(2) / math.log(i+2)
    return 0

Initializing the models

In [24]:
# Build models
gmf_model = get_GMF_model(num_users, num_items, num_factors, regs = [[0,0]])
mlp_model = get_MLP_model(num_users, num_items, num_factors, regs = [[0,0],0,0,0], layers = [32,16,8])
nmf_model = get_NMF_model(num_users, num_items, latent_dim_GMF=num_factors, latent_dim_MLP=num_factors, reg_GMF=[[0,0]],
                      regs_MLP=[[0,0],0,0,0], layers=[32,16,8])

gmf_model.compile(optimizer=Adam(lr=learning_rate), loss='binary_crossentropy')
mlp_model.compile(optimizer=Adam(lr=learning_rate), loss='binary_crossentropy')
nmf_model.compile(optimizer=Adam(lr=learning_rate), loss='binary_crossentropy')

**Init performances**

In [25]:
models = [('GMF', gmf_model), ('MLP', mlp_model), ('NMF', nmf_model)]
def initPerformance():
  # Init performance
  for name, model in models:
    start = time()
    (mrrs, ndcgs) = evaluate_model(model, test_rating, test_negative, topK)
    mrr, ndcg = np.array(mrrs).mean(), np.array(ndcgs).mean()
    print(f'{name} Init: MRR = {mrr}, NDCG =  {ndcg}, Time (minutes):  {(time()-start)/60}')

In [26]:
initPerformance()

GMF Init: MRR = 0.027220316934720906, NDCG =  0.04289594653594933, Time (minutes):  2.9573014775911965
MLP Init: MRR = 0.028513612950699044, NDCG =  0.04417181337208131, Time (minutes):  2.9358290076255797
NMF Init: MRR = 0.03379802638494691, NDCG =  0.05189504976179986, Time (minutes):  2.9414419452349345


**Training and evaluating performances**

In [27]:
def train_evaluate_models(epochs=10):
  # Train model
  for name, model in models:
    for topK in [5, 10]:
      print(f'\n{name} Model: cutoff = {topK}\n')
      for epoch in range(1, epochs + 1):

        start = time()
        # Generate training instances
        user_input, item_input = users, items

        # Training
        hist = model.fit([np.array(user_input), np.array(item_input)], #input
                          np.array(labels), # labels 
                          batch_size=batch_size, epochs=1, verbose=0, shuffle=True)
        loss = hist.history['loss'][0]
        if epoch % 5 == 0:
          # Evaluation
          (mrrs, ndcgs) = evaluate_model(model, test_rating, test_negative, topK)

          end = time()

          mrr, ndcg = np.array(mrrs).mean(), np.array(ndcgs).mean() 
          print(f'[Epoch: {epoch}, Time (minutes): {(end-start) / 60}, Test MRR = {mrr}, Test NDCG = {ndcg}, Loss = {loss}]')

        else:
          print(f'[Epoch: {epoch}, Loss = {loss}]')


In [28]:
train_evaluate_models()


GMF Model: cutoff = 5

[Epoch: 1, Loss = 0.3628537058830261]
[Epoch: 2, Loss = 0.307639479637146]
[Epoch: 3, Loss = 0.28916656970977783]
[Epoch: 4, Loss = 0.27786773443222046]
[Epoch: 5, Time (minutes): 3.693118929862976, Test MRR = 0.2447488962472406, Test NDCG = 0.2913974783392471, Loss = 0.27368664741516113]
[Epoch: 6, Loss = 0.2717829644680023]
[Epoch: 7, Loss = 0.27060410380363464]
[Epoch: 8, Loss = 0.2698017656803131]
[Epoch: 9, Loss = 0.2691515386104584]
[Epoch: 10, Time (minutes): 3.6946535229682924, Test MRR = 0.25068708609271523, Test NDCG = 0.2990442014220135, Loss = 0.26864802837371826]

GMF Model: cutoff = 10

[Epoch: 1, Loss = 0.26818034052848816]
[Epoch: 2, Loss = 0.2678431272506714]
[Epoch: 3, Loss = 0.2674868702888489]
[Epoch: 4, Loss = 0.2671961784362793]
[Epoch: 5, Time (minutes): 3.8103481690088907, Test MRR = 0.2761968490486702, Test NDCG = 0.3581624982618208, Loss = 0.2669491171836853]
[Epoch: 6, Loss = 0.2666969895362854]
[Epoch: 7, Loss = 0.26646482944488525]
[

**Comparsion:**

*   First of all, as we can see, for all three models, we got better results when using a cutoff of 10 comparing to when we used a cutoff of 5. This is true regarding both MRR and NDCG metrics. 
*   When comparing between the three models (MLP, GMF, NMF), we can see that the best results achieved when using NMF model (this is true for both MRR and NDCG metrics).
* The results for MLP and GMF are quite similar. GMF got better result w.r.t MRR metric, and MLP got better results w.r.t to NDCG metric.
* For all three models, we can clearly see that the learning process is more efficient when using a cutoff = 5. That is, we are having a better improvement  relative to the starting point (in terms of loss) and we manage to decrease the loss more quickly and intensively with this cutoff.




# 2.c

Both the MRR and NDCG metrics are used as a measures for evaluating the ordered output of a model. The order of the output (the list of items) is by most relevant to the least relevant item. Since in this exercise we use an implicit ranking, there is not such thing as “more relevant” and “less relevant”, because every items that the user has ranked we consider as relevant (‘1’) and every items that was not been ranked we consider as not relevant (‘0’). In conclusion, using the MRR and NDCG metrics for evaluating measures in the implicit case is less accurate since the basic assumption - that the order of the items are relevant - does not hold.

# 2.d

We suggest the following method for representing an item and measuring item similarity using NeuMF model: for every user u and item i, use the NeuMF model which outputs y_ui. For an item I create a vector which contains all the outputs y_ui for all the users u in U (set of users), in a predefined order. The, for two items i_1 and i_2, we will use their vectors to calculate their cosine similarity score. We do expect this method to be a reliable way to measure items similarity, because the basic assumption is that similar items will get similar relevancy score for each user, so their vectors (or their angle) will be similar, resulting in high cosine similary score (close to 1). 

#3.a

We will define the NMF model from new, with RELU as last activation function and loss set to MSE loss.

In [29]:
def get_NMF_model_2(num_users, num_items, latent_dim_GMF, latent_dim_MLP, reg_GMF=[[0,0]], regs_MLP=[[0,0],0,0], layers=[20,10]):
    #Neural matrix factorization
    assert len(layers) + 1 == len(regs_MLP), 'the number of regs is equal to number of layers + the embedding layer'
    num_layer = len(layers) #Number of layers in the MLP

    # Input variables
    user_input = Input(shape=(1,), dtype='int32', name = 'user_input')
    item_input = Input(shape=(1,), dtype='int32', name = 'item_input')
    
    # Embedding layer
    MF_Embedding_User = Embedding(input_dim = num_users, output_dim = latent_dim_GMF, name = 'MF_user_embedding',
                                   embeddings_regularizer = l2(reg_GMF[0][0]), input_length=1,embeddings_initializer=RandomNormal(mean=0.0, stddev=0.01)) #init = init_normal,
    MF_Embedding_Item = Embedding(input_dim = num_items, output_dim = latent_dim_GMF, name = 'MF_item_embedding',
                                   embeddings_regularizer = l2(reg_GMF[0][1]), input_length=1,embeddings_initializer=RandomNormal(mean=0.0, stddev=0.01))  #init = init_normal, 
    
    MLP_Embedding_User = Embedding(input_dim = num_users, output_dim = latent_dim_MLP, name = 'MLP_user_embedding',
                                   embeddings_regularizer = l2(regs_MLP[0][0]), input_length=1,embeddings_initializer=RandomNormal(mean=0.0, stddev=0.01)) #init = init_normal,
    MLP_Embedding_Item = Embedding(input_dim = num_items, output_dim = latent_dim_MLP, name = 'MLP_item_embedding',
                                   embeddings_regularizer = l2(regs_MLP[0][1]), input_length=1,embeddings_initializer=RandomNormal(mean=0.0, stddev=0.01)) #init = init_normal,
    
    # MF part
    mf_user_latent = Flatten()(MF_Embedding_User(user_input))
    mf_item_latent = Flatten()(MF_Embedding_Item(item_input))
    mf_vector = Multiply()([mf_user_latent, mf_item_latent]) #merge([mf_user_latent, mf_item_latent], mode = 'mul') # element-wise multiply

    # MLP part 
    mlp_user_latent = Flatten()(MLP_Embedding_User(user_input))
    mlp_item_latent = Flatten()(MLP_Embedding_Item(item_input))
    mlp_vector = Concatenate(axis=-1)([mlp_user_latent, mlp_item_latent])#merge([mlp_user_latent, mlp_item_latent], mode = 'concat')
    for idx in range(num_layer):
        layer =  Dense(layers[idx], kernel_regularizer = l2(regs_MLP[idx+1]), activation='tanh', name = 'layer%d' %idx)
        mlp_vector = layer(mlp_vector)

    # Concatenate MF and MLP parts
    predict_vector = Concatenate(axis=-1)([mf_vector, mlp_vector])
    
    # Final prediction layer
    # Changed from sigmoid to RELU.
    prediction = Dense(1, activation='relu', kernel_initializer='lecun_uniform', name = "prediction")(predict_vector)    
    model = Model(inputs=[user_input, item_input], outputs=prediction)
    
    return model

Initialize the model

In [30]:
nmf_model_2 = get_NMF_model_2(num_users, num_items, latent_dim_GMF=num_factors, latent_dim_MLP=num_factors, reg_GMF=[[0,0]],
                      regs_MLP=[[0,0],0,0,0], layers=[32,16,8])
nmf_model_2.compile(optimizer=Adam(lr=learning_rate), loss='mse')
print(nmf_model_2.summary())

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
item_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
MLP_user_embedding (Embedding)  (None, 1, 8)         48320       user_input[0][0]                 
__________________________________________________________________________________________________
MLP_item_embedding (Embedding)  (None, 1, 8)         29648       item_input[0][0]                 
____________________________________________________________________________________________

Learning for 10 epochs (as before) and outputting the metrics evaluation.

In [31]:
print(f'\nNMF Model (with RELU and MSE loss):\n')
for topK in [5, 10]:
  print(f'\ncutoff = {topK}\n')
  for epoch in range(1, 11):

    start = time()
    # Generate training instances
    user_input, item_input = users, items

    # Training
    hist = nmf_model_2.fit([np.array(user_input), np.array(item_input)], #input
                      np.array(labels), # labels 
                      batch_size=batch_size, epochs=1, verbose=0, shuffle=True)
    loss = hist.history['loss'][0]
    if epoch % 5 == 0:
      # Evaluation
      (mrrs, ndcgs) = evaluate_model(nmf_model_2, test_rating, test_negative, topK)

      end = time()

      mrr, ndcg = np.array(mrrs).mean(), np.array(ndcgs).mean() 
      print(f'[Epoch: {epoch}, Time (minutes): {(end-start) / 60}, Test MRR = {mrr}, Test NDCG = {ndcg}, Loss = {loss}]')

    else:
      print(f'[Epoch: {epoch}, Loss = {loss}]')


NMF Model (with RELU and MSE loss):


cutoff = 5

[Epoch: 1, Loss = 0.09962603449821472]
[Epoch: 2, Loss = 0.08714807033538818]
[Epoch: 3, Loss = 0.08440519124269485]
[Epoch: 4, Loss = 0.0829232707619667]
[Epoch: 5, Time (minutes): 3.9858445564905804, Test MRR = 0.2618846578366446, Test NDCG = 0.3107282478360862, Loss = 0.08188372105360031]
[Epoch: 6, Loss = 0.08101693540811539]
[Epoch: 7, Loss = 0.08027520775794983]
[Epoch: 8, Loss = 0.07961497455835342]
[Epoch: 9, Loss = 0.07904289662837982]
[Epoch: 10, Time (minutes): 3.9726574778556825, Test MRR = 0.2679470198675497, Test NDCG = 0.31796025532737815, Loss = 0.07855463773012161]

cutoff = 10

[Epoch: 1, Loss = 0.07813173532485962]
[Epoch: 2, Loss = 0.07780124247074127]
[Epoch: 3, Loss = 0.07747158408164978]
[Epoch: 4, Loss = 0.07721131294965744]
[Epoch: 5, Time (minutes): 4.016893311341604, Test MRR = 0.28702045884578997, Test NDCG = 0.371383434170011, Loss = 0.07695277780294418]
[Epoch: 6, Loss = 0.07672039419412613]
[Epoch: 7, Los

As we can see, we got very similar results to the ones we got with the original NMF implementation. We get almost the same scores both for NDCG and MRR metrics, for both settings of the cutoff (5, 10). Of course the loss is different, but this is because we use a different loss function (MSE). In general, sigmoid function restricts each neuron to be in (0,1), which may limit the model’s performance; and it is known to suffer from saturation, where neurons stop learning when their output is near either 0 or 1. RELU on the other hand, was proven to be non-saturated. Moreover, it encourages sparse activations, being well-suited for sparse data and making the model less likely to be overfitting.